In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
train_data_dir = './archive/train'
val_data_dir = './archive/valid'

img_width, img_height = 150, 150

train_transforms = transforms.Compose([
    transforms.Resize((img_width, img_height)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((img_width, img_height)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:

train_dataset = datasets.ImageFolder(train_data_dir, transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


val_dataset = datasets.ImageFolder(val_data_dir, transform=val_transforms)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:

class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 18 * 18, 128),  
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:
model = CNN(num_classes=len(train_dataset.classes))

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

epochs = 10  

In [ ]:
for epoch in range(epochs):
    # Training
    model.train()
    train_loss = 0.0
    train_correct = 0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_correct += (predicted == labels).sum().item()

    train_loss = train_loss / len(train_dataset)
    train_acc = train_correct / len(train_dataset)
    val_loss = val_loss / len(val_dataset)
    val_acc = val_correct / len(val_dataset)

    print(f'Epoch {epoch+1}/{epochs}: '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

Epoch 1/10: Train Loss: 3.9736, Train Acc: 0.0863, Val Loss: 3.4696, Val Acc: 0.1680
Epoch 2/10: Train Loss: 3.2471, Train Acc: 0.2038, Val Loss: 3.0564, Val Acc: 0.2360
Epoch 3/10: Train Loss: 2.8121, Train Acc: 0.2937, Val Loss: 2.6065, Val Acc: 0.3480
Epoch 4/10: Train Loss: 2.4498, Train Acc: 0.3730, Val Loss: 2.4155, Val Acc: 0.3900
Epoch 5/10: Train Loss: 2.1983, Train Acc: 0.4240, Val Loss: 2.3295, Val Acc: 0.4160
Epoch 6/10: Train Loss: 1.9962, Train Acc: 0.4709, Val Loss: 2.1491, Val Acc: 0.4440
Epoch 7/10: Train Loss: 1.8063, Train Acc: 0.5124, Val Loss: 2.1594, Val Acc: 0.4460
Epoch 8/10: Train Loss: 1.6501, Train Acc: 0.5488, Val Loss: 2.2736, Val Acc: 0.4360
Epoch 9/10: Train Loss: 1.5075, Train Acc: 0.5833, Val Loss: 2.2487, Val Acc: 0.4540
Epoch 10/10: Train Loss: 1.3909, Train Acc: 0.6178, Val Loss: 2.2349, Val Acc: 0.4720


In [7]:
torch.save(model.state_dict(), 'sports_classification_model.pth')